In [1]:
import numpy as np
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import matplotlib.pyplot as plt
import pandas as pd
from identification import *

plt.style.use('default')

# Validação dos modelos

In [2]:
df_data_n11 = pd.read_csv('../data/entrada_saida_nsenos_n11_111v.csv', header=None)

df_theta_hat_2o = pd.read_csv('../data/theta_hat_2o_emax_111.csv', header=None)
df_theta_hat_ed = pd.read_csv('../data/theta_hat_ed_emax_111.csv', header=None)

In [3]:
data_n11  = np.array(df_data_n11    , ndmin=2)
u_ar_n11  = np.array(df_data_n11[0] , ndmin=2).T/11.1
y_ar_n11  = np.array(df_data_n11[1] , ndmin=2).T

theta_hat_2o = np.array(df_theta_hat_2o, ndmin=2)
theta_hat_ed = np.array(df_theta_hat_ed, ndmin=2)

In [4]:
nz = number_of_zeros(y_ar_n11)
y_ar_n11 = y_ar_n11[nz-1:]
u_ar_n11 = u_ar_n11[nz-1:]

In [5]:
u_ar_n11.size

1199

In [6]:
print(y_ar_n11[:2].T)

[[  0.         151.36558196]]


In [7]:
y_hat_2o_n11 =         estimate_output(y=y_ar_n11,u=u_ar_n11,theta=theta_hat_2o, ord=2)
y_hat_ed_n11 = estimate_output_ed_fesc(y=y_ar_n11,u=u_ar_n11,theta=theta_hat_ed)

In [8]:
T1 = 0
T2 = -1

V_2o = cost_func(y_ar_n11[T1:T2], y_hat_2o_n11[T1:T2])
V_ed = cost_func(y_ar_n11[T1:T2], y_hat_ed_n11[T1:T2])


print('Resultados validação n=11:')

print(f'ARX 2ª ordem V = {V_2o}')
print(f'discretizado V = {V_ed}')

Resultados validação n=11:
ARX 2ª ordem V = 4377.360720406043
discretizado V = 1677.635277487421


# Análise dos resíduos

In [9]:
# e_t = y_t - y_hat_t
e_2o_n11_d = y_ar_n11 - y_hat_2o_n11
e_ed_n11_d = y_ar_n11 - y_hat_ed_n11

In [10]:
df = pd.DataFrame(e_2o_n11_d)
df.to_csv('../data/e_2o_n11_d.csv',header=False,index=False)
df = pd.DataFrame(e_ed_n11_d)
df.to_csv('../data/e_ed_n11_d.csv',header=False,index=False)

In [11]:
# função de covariança como mostrada no livro
def prev_cov(e,u,tau_size):
    u_m = np.mean(u)
    e_m = np.mean(e)
    Tau = np.arange(tau_size)
    R = np.zeros(Tau.shape)
    for tau in Tau:
        N = np.arange(tau,u.size)
        for t in N:
            R[tau] += ((e[t]-e_m) * (u[t-tau]-u_m))
    return R / u.size

In [ ]:
Y_ed, phi = build_Y_phi_ed_load_fesc(u_ar_n11,y_ar_n11)
yk2_yk1 = -phi[:,2]
yk22 = -phi[:,3]

In [12]:
T1 = 20
# e * e
R_e_ed = prev_cov(e_ed_n11_d[T1:],e_ed_n11_d[T1:],tau_size=u_ar_n11.size)
# e * u
R_eu_ed = prev_cov(e_ed_n11_d[T1:],u_ar_n11[T1:],tau_size=u_ar_n11.size)
R_u_ed = prev_cov(u_ar_n11[T1:],u_ar_n11[T1:],tau_size=1)
# e * y[k-2]y[k-1]
R_eyk2yk1_ed = prev_cov(e_ed_n11_d[T1:],yk2_yk1[T1:],tau_size=e_ed_n11_d.size)
R_yk2yk1_ed = prev_cov(yk2_yk1[T1:],yk2_yk1[T1:],tau_size=1)
# e * y[k-2]**2
R_eyk22_ed = prev_cov(e_ed_n11_d[T1:], yk22[T1:],tau_size=e_ed_n11_d.size)
R_yk22_ed = prev_cov(yk22[T1:],yk22[T1:],tau_size=1)
# e * u**2
R_eu2_ed = prev_cov(e_ed_n11_d[T1:],u_ar_n11[T1:]**2,tau_size=u_ar_n11.size)
R_u2_ed = prev_cov(u_ar_n11[T1:]**2,u_ar_n11[T1:]**2,tau_size=1)

In [ ]:
corr_e_e = R_e_ed/np.abs(R_e_ed[0])

corr_e_u = R_eu_ed/np.sqrt(R_e_ed[0]*R_u_ed)

corr_e_yk2yk1 = R_eyk2yk1_ed/np.sqrt(R_e_ed[0]*R_yk2yk1_ed)

corr_e_yk22 = R_eyk22_ed/np.sqrt(R_e_ed[0]*R_yk22_ed)

corr_e_u2 = R_eu2_ed/np.sqrt(R_e_ed[0]*R_u2_ed)

In [ ]:
print(corr_e_e )

print(corr_e_u )

print(corr_e_yk2yk1 )

print(corr_e_yk22 )

print(corr_e_u2 )

In [ ]:
lim = 1.96/np.sqrt(u_ar_n11.size)
print(lim)

In [ ]:
t = np.arange(0,R_e_ed.size)*0.1024

plt.figure(1, figsize=[15,5])
plt.grid(True)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.xlabel('Delay, $\\tau$ (s)',fontsize=25)
plt.ylabel("Estimate of the\nautocorrelation function\nbetween residuals, $\hat{\delta}_{\\varepsilon \\varepsilon}^N(\\tau)$",fontsize=25)
plt.plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
plt.plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
plt.plot(t,corr_e_e, label='$\hat{\delta}_{\\varepsilon \\varepsilon}^N(\\tau)$')
#plt.xlim([0,20])
plt.legend(fontsize=20, ncol=2)

plt.figure(2, figsize=[15,5])
plt.grid(True)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.xlabel('Delay, $\\tau$ (s)',fontsize=25)
plt.ylabel("Estimate of the correlation\nfunction between residuals\nand past inputs, $\hat{\delta}_{\\varepsilon u}^N(\\tau)$",fontsize=25)
plt.plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
plt.plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
plt.plot(t,corr_e_u, label='$\hat{\delta}_{\\varepsilon u}^N(\\tau)$')
plt.legend(fontsize=20, ncol=2)

plt.figure(3, figsize=[15,5])
plt.grid(True)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.xlabel('Delay, $\\tau$ (s)',fontsize=25)
plt.ylabel("Estimate of the correlation\nfunction between residuals\nand HOT, $\hat{\delta}_{\\varepsilon y(k-1)y(k-2)}^N(\\tau)$",fontsize=25)
plt.plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
plt.plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
plt.plot(t,corr_e_yk2yk1, label='$\hat{\delta}_{\\varepsilon y(k-1)y(k-2)}^N(\\tau)$')
plt.legend(fontsize=20, ncol=2)

plt.figure(4, figsize=[15,5])
plt.grid(True)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.xlabel('Delay, $\\tau$ (s)',fontsize=25)
plt.ylabel("Estimate of the correlation\nfunction between residuals\nand HOT, $\hat{\delta}_{\\varepsilon y^2(k-2)}^N(\\tau)$",fontsize=25)
plt.plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
plt.plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
plt.plot(t,corr_e_yk22, label='$\hat{\delta}_{\\varepsilon y^2(k-2)}^N(\\tau)$')
plt.legend(fontsize=20, ncol=2)

plt.figure(5, figsize=[15,5])
plt.grid(True)
plt.xticks(fontsize='xx-large')
plt.yticks(fontsize='xx-large')
plt.xlabel('Delay, $\\tau$ (s)',fontsize=25)
plt.ylabel("Estimate of the correlation\nfunction between residuals\nand HOT, $\hat{\delta}_{\\varepsilon u^2}^N(\\tau)$",fontsize=25)
plt.plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
plt.plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
plt.plot(t,corr_e_u2, label='$\hat{\delta}_{\\varepsilon u^2}^N(\\tau)$')
plt.legend(fontsize=20, ncol=2)

In [ ]:
t = np.arange(0,R_e_ed.size)*0.1024

fig, axs = plt.subplots(5, 1, figsize=[15,33], sharex=True)

axs[0].grid(True)
axs[1].grid(True)
axs[2].grid(True)
axs[3].grid(True)
axs[4].grid(True)

axs[0].tick_params(labelsize='xx-large')
axs[1].tick_params(labelsize='xx-large')
axs[2].tick_params(labelsize='xx-large')
axs[3].tick_params(labelsize='xx-large')
axs[4].tick_params(labelsize='xx-large')

axs[0].set_title('(a)', fontsize='xx-large')
axs[1].set_title('(b)', fontsize='xx-large')
axs[2].set_title('(c)', fontsize='xx-large')
axs[3].set_title('(d)', fontsize='xx-large')
axs[4].set_title('(e)', fontsize='xx-large')


#axs[0].set_xlabel('Delay, $\\tau$ (s)',fontsize=25)
axs[0].set_ylabel("Estimate of the\nautocorrelation function\nbetween residuals, $\hat{\delta}_{\\varepsilon \\varepsilon}^N(\\tau)$",fontsize=25)
axs[0].plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
axs[0].plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
axs[0].plot(t,corr_e_e, label='$\hat{\delta}_{\\varepsilon \\varepsilon}^N(\\tau)$')
axs[0].legend(fontsize=20, ncol=2)


#axs[1].set_xlabel('Delay, $\\tau$ (s)',fontsize=25)
axs[1].set_ylabel("Estimate of the correlation\nfunction between residuals\nand past inputs, $\hat{\delta}_{\\varepsilon u}^N(\\tau)$",fontsize=25)
axs[1].plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
axs[1].plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
axs[1].plot(t,corr_e_u, label='$\hat{\delta}_{\\varepsilon u}^N(\\tau)$')
axs[1].legend(fontsize=20, ncol=2)


#axs[2].set_xlabel('Delay, $\\tau$ (s)',fontsize=25)
axs[2].set_ylabel("Estimate of the correlation\nfunction between residuals\nand HOT, $\hat{\delta}_{\\varepsilon y(k-1)y(k-2)}^N(\\tau)$",fontsize=25)
axs[2].plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
axs[2].plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
axs[2].plot(t,corr_e_yk2yk1, label='$\hat{\delta}_{\\varepsilon y(k-1)y(k-2)}^N(\\tau)$')
axs[2].legend(fontsize=20, ncol=2)


#axs[3].set_xlabel('Delay, $\\tau$ (s)',fontsize=25)
axs[3].set_ylabel("Estimate of the correlation\nfunction between residuals\nand HOT, $\hat{\delta}_{\\varepsilon y^2(k-2)}^N(\\tau)$",fontsize=25)
axs[3].plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
axs[3].plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
axs[3].plot(t,corr_e_yk22, label='$\hat{\delta}_{\\varepsilon y^2(k-2)}^N(\\tau)$')
axs[3].legend(fontsize=20, ncol=2)


axs[4].set_xlabel('Delay, $\\tau$ (s)',fontsize=25)
axs[4].set_ylabel("Estimate of the correlation\nfunction between residuals\nand HOT, $\hat{\delta}_{\\varepsilon u^2}^N(\\tau)$",fontsize=25)
axs[4].plot(t, lim*np.ones(corr_e_e.shape), color='gray', label='$\pm 1.96/\sqrt{N}$')
axs[4].plot(t,-lim*np.ones(corr_e_e.shape), color='gray')
axs[4].plot(t,corr_e_u2, label='$\hat{\delta}_{\\varepsilon u^2}^N(\\tau)$')
axs[4].legend(fontsize=20, ncol=2)